In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import numpy as np 

In [2]:
def data_normalize(data_temp):
    data_temp2=data_temp.T
    data_temp2 -=np.mean(data_temp2,axis=0)
    data_temp2 /=np.std(data_temp2,axis=0)
    data_temp=data_temp2.T
    return data_temp

In [3]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [4]:
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [5]:
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial_Signals/'
    # load all 9 files as a single array
    filenames = list()
    # body acceleration
    filenames += ["acc_x.txt","acc_y.txt","acc_z.txt"]
    # body gyroscope
    filenames += ["gyr_x.txt","gyr_y.txt","gyr_z.txt"]
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [6]:
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'Dataset_4/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'Dataset_4/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [7]:
trainX, trainy, testX, testy=load_dataset()

(35373, 128, 6) (35373, 1)
(3941, 128, 6) (3941, 1)
(35373, 128, 6) (35373, 20) (3941, 128, 6) (3941, 20)


In [8]:
trainX.transpose(0,2,1)

array([[[-5.35942e+00, -3.94684e+00, -2.23738e+00, ..., -4.72256e+00,
         -4.22457e+00, -4.67468e+00],
        [-8.03015e+00, -5.47314e+00, -5.92804e+00, ..., -1.12240e+01,
         -1.26127e+01, -1.17029e+01],
        [ 4.17549e+00,  4.50110e+00,  3.08373e+00, ..., -4.97994e-01,
         -3.36146e+00, -3.84988e+00],
        [ 1.05461e-01,  4.77239e-01,  5.08131e-01, ...,  7.90426e-01,
          6.83900e-01, -3.62190e-01],
        [ 2.47994e+00,  2.85704e+00,  1.29856e+00, ..., -4.60194e-01,
         -1.06740e+00,  2.76969e-01],
        [ 4.92152e-01,  4.28236e-01,  1.72573e-01, ...,  1.59790e-01,
          4.38889e-01,  2.92948e-01]],

       [[-5.37498e-01,  3.43688e+00,  6.07050e+00, ..., -2.83713e-01,
          1.06662e+00, -9.34936e-01],
        [-1.69270e+01, -1.62614e+01, -1.16789e+01, ..., -9.11712e+00,
         -7.99184e+00, -7.86735e+00],
        [-1.02615e+01, -1.67594e+00,  2.55701e+00, ..., -1.74777e+00,
          9.09797e-02, -3.06458e-01],
        [-9.18258e-01, -9.

In [9]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('trainX.npy', trainX)
#np.load(tranx.npy)

In [10]:
# load numpy array from npy file
from numpy import load
# load array
trainX = load('trainX.npy')
# print the array
print(trainX)

[[[-5.35942e+00 -8.03015e+00  4.17549e+00  1.05461e-01  2.47994e+00
    4.92152e-01]
  [-3.94684e+00 -5.47314e+00  4.50110e+00  4.77239e-01  2.85704e+00
    4.28236e-01]
  [-2.23738e+00 -5.92804e+00  3.08373e+00  5.08131e-01  1.29856e+00
    1.72573e-01]
  ...
  [-4.72256e+00 -1.12240e+01 -4.97994e-01  7.90426e-01 -4.60194e-01
    1.59790e-01]
  [-4.22457e+00 -1.26127e+01 -3.36146e+00  6.83900e-01 -1.06740e+00
    4.38889e-01]
  [-4.67468e+00 -1.17029e+01 -3.84988e+00 -3.62190e-01  2.76969e-01
    2.92948e-01]]

 [[-5.37498e-01 -1.69270e+01 -1.02615e+01 -9.18258e-01 -1.43065e+00
    1.56594e-01]
  [ 3.43688e+00 -1.62614e+01 -1.67594e+00 -9.71521e-01 -8.78843e-01
   -1.16114e-01]
  [ 6.07050e+00 -1.16789e+01  2.55701e+00  3.73908e-01  1.38804e+00
   -7.25445e-01]
  ...
  [-2.83713e-01 -9.11712e+00 -1.74777e+00 -1.81095e-02  6.81769e-02
   -2.32228e-01]
  [ 1.06662e+00 -7.99184e+00  9.09797e-02  3.27036e-01 -4.14388e-01
   -4.83630e-01]
  [-9.34936e-01 -7.86735e+00 -3.06458e-01 -2.52468e

In [11]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('testX.npy', testX)
#np.load(tranx.npy)
# load numpy array from npy file
from numpy import load
# load array
testX = load('testX.npy')
# print the array
print(testX)

[[[-4.77045e-01 -7.92840e+00  1.14084e+00 -4.85638e-02  1.36529e-01
    7.20821e-02]
  [ 1.08757e+00 -8.68078e+00  1.20728e+00  2.00364e-01 -4.88690e-03
    1.51189e-01]
  [ 2.91494e-01 -9.47026e+00  1.66996e-01  3.28645e-01 -2.79471e-01
   -8.39940e-02]
  ...
  [ 4.68066e-01 -9.64983e+00 -1.56162e+00  7.94125e-02 -1.21562e-01
   -4.55095e-02]
  [-1.31083e-01 -9.85872e+00 -1.01335e+00 -5.52833e-02 -1.27976e-01
   -2.71835e-02]
  [ 9.27753e-02 -1.01359e+01 -3.72897e-01 -1.44775e-01  1.76235e-01
    1.55771e-02]]

 [[ 3.79601e+00 -3.62961e+00 -5.36301e+00 -3.94148e-01 -1.74171e+00
   -4.15453e-01]
  [-4.46519e-01 -4.58250e+00  1.12049e+00  1.33158e+00 -5.35828e-01
    1.70442e-01]
  [-1.07859e+00 -3.30400e+00 -6.08127e-01  7.76578e-01  5.95483e-01
   -1.59790e-02]
  ...
  [-2.84072e+00 -9.35654e+00  1.91536e-02  5.79504e-01 -1.03650e+00
    3.74973e-01]
  [-1.10253e+00 -1.02807e+01  2.05901e-01  4.00539e-01 -1.31027e-01
    4.90022e-02]
  [-2.31040e-01 -1.12623e+01 -1.71904e+00  2.71643e

In [12]:
# save numpy array as npy file
from numpy import asarray
from numpy import save
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to npy file
save('testy.npy', testy)
#np.load(tranx.npy)
# load numpy array from npy file
from numpy import load
# load array
testy = load('testy.npy')
# print the array
print(testy)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
